In [1]:
import os, sys; [(sys.path.append(d),)
                 for d in (os.path.abspath(os.path.join(os.getcwd(), *([".."] * i))) 
                           for i in range(len(os.getcwd().split(os.sep)))) 
                           if os.path.isfile(os.path.join(d, 'main.py'))]

[(None,)]

In [2]:
import argparse
import math
import pandas as pd
import torch
import torch.nn.functional as F
from torch.nn import Linear, MSELoss
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader, Subset
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
from torch_geometric.nn.models.tgn import TGNMemory, IdentityMessage, LastAggregator
from torch_geometric.nn.conv import TransformerConv

In [3]:
from Model.Prediction.tgn_recommendation import haversine, load_and_preprocess, recommend, MyTGN

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
(source, dest, edge_attr, norm_ts,
    user_df, biz_df,
    user_to_index, business_to_index,
    user_feats, biz_feats, avg_rating) = load_and_preprocess(data_directory="../../Data")
total_nodes = len(user_to_index) + len(business_to_index)


In [ ]:

model = MyTGN(total_nodes, user_feats.size(1), biz_feats.size(1)).to(device)
model.load_state_dict(torch.load('../Saved_Models/tgn_model.pt', weights_only=True))
model.eval()


In [36]:

# sample recommend
uid = 'AUi8MPWJ0mLkMfwbui27lg'
long = -75.155564
lat = 39.955505
radius = 100
hour = 10
recs = recommend(model, uid, lat, long, radius, hour,
                    user_to_index, business_to_index, biz_df,
                    top_k=20, device=device, avg_rating=avg_rating, user_feats=user_feats, biz_feats=biz_feats)
print("Top Recommendations:")
for bid, sc in recs:
    name = biz_df.loc[biz_df['business_id'] == bid, 'name'].values[0]
    print(f"  {name}: {sc:.8f}")

Top Recommendations:
  @Ramen: 3.80472040
  Phils Appliances: 3.80436683
  Little Kitchen: 3.80345440
  DB Cleaning Services: 3.80344272
  WYSP 94.1 FM: 3.80339575
  Balcony Bar @ The Kimmel Center: 3.80307007
  Grovers Meat Market: 3.80305052
  Crust Vegan Bakery: 3.80293918
  Dollar Tree: 3.80288887
  The Parking Spot: 3.80284071
  Popeyes Louisiana Kitchen: 3.80277896
  Quick & Clean Coin Laundry: 3.80245185
  Original 13 Ciderworks: 3.80243778
  Desimone Auto Group: 3.80242991
  Blatt Tire & Auto Repair: 3.80240750
  Tequilas Restaurant: 3.80238724
  Pete's BBQ Breakfast & Deli: 3.80238438
  Wawa: 3.80238366
  Emma's Nails And Spa: 3.80236244
  Just Salad: 3.80235863
